In [3]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
import random
import json



In [4]:
url = "https://boardgamegeek.com/users?country=Vietnam&state=&city="
response = requests.get(url)
soup = bs(response.text, "html.parser")
country = soup.findAll("option")
countries = [c.text for c in country]
countries = [country.replace(" ", "%20") for country in countries]


In [5]:
#We assign weights to each country depending on the number of pages in boardgamegeek per user

def get_weight(country):
    url = f"https://boardgamegeek.com/users?country={country}&state=&city="
    # Fetch the page to get the number of pages for the country
    response = requests.get(url)
    soup = bs(response.text, 'html.parser')
    last_page_link = soup.find('a', title="last page")
    if last_page_link:
            last_page_number = int(last_page_link.text.strip('[]'))
    else:
        last_page_number = 1
    
    return last_page_number
            
def createWeights(countries):
    weights = {}
    for country in countries:
        weight = get_weight(country)
        weights[country] = weight
    return weights


    


In [6]:

weights = createWeights(countries)

In [7]:
with open('countries.json', 'w') as json_file:
    json.dump(weights, json_file, indent=4)

In [17]:
country_list = list(weights.keys())
weightsC = list(weights.values())

# Randomly choose a country based on weights
chosen_country = random.choices(country_list, weights=weightsC, k=1)[0]
chosen_country

'United%20Kingdom'

In [18]:
import requests
from bs4 import BeautifulSoup
import random
import time

In [22]:
usernames = []

# Open a file to write usernames
with open('username2s.txt', 'a') as file:
    while len(usernames) < 300000:
        country = random.choices(country_list, weights=weightsC, k=1)[0]
        
        
        last_page_number = weights[country]
        random_page = random.randint(1, last_page_number)
        
        
        # Fetch a random page
        page_url = f"https://boardgamegeek.com/users/page/{random_page}?country={country}&state=&city="
        response = requests.get(page_url)
        if response.status_code==200:
            page_soup = BeautifulSoup(response.text, 'html.parser')
            
            # Extract usernames
            user_divs = page_soup.find_all('div', class_='username')
            for div in user_divs:
                username = div.find('a').text
                if username not in usernames:
                    usernames.append(username)
                    file.write(username + '\n')
                    if len(usernames) >= 300000:
                        break
        
        print("Added usernames:", len(usernames))
            
        # Wait a bit before making another request

# The 'usernames.txt' file now contains up to 1000 unique usernames.



Added usernames: 25
Added usernames: 50
Added usernames: 75
Added usernames: 100
Added usernames: 125
Added usernames: 150
Added usernames: 175
Added usernames: 200
Added usernames: 225
Added usernames: 250
Added usernames: 275
Added usernames: 300
Added usernames: 325
Added usernames: 350
Added usernames: 375
Added usernames: 400
Added usernames: 425
Added usernames: 450
Added usernames: 475
Added usernames: 500
Added usernames: 525
Added usernames: 550
Added usernames: 575
Added usernames: 600
Added usernames: 625
Added usernames: 650
Added usernames: 675
Added usernames: 700
Added usernames: 725
Added usernames: 750
Added usernames: 775
Added usernames: 800
Added usernames: 825
Added usernames: 850
Added usernames: 875
Added usernames: 900
Added usernames: 925
Added usernames: 950
Added usernames: 975
Added usernames: 1000
Added usernames: 1025
Added usernames: 1050
Added usernames: 1075
Added usernames: 1100
Added usernames: 1125
Added usernames: 1150
Added usernames: 1175
Added us

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [9]:
import random
import requests
from bs4 import BeautifulSoup
from joblib import Parallel, delayed
import time
import threading


country_list = list(weights.keys())
weightsC = list(weights.values())

# Thread lock for writing to the file
lock = threading.Lock()

usernames = []
total_usernames_acquired = 0

def fetch_usernames():
    local_usernames = []
    while len(local_usernames) < 100000:  # Adjust this to control the number of usernames per parallel task
        country = random.choices(country_list, weights=weightsC, k=1)[0]
        
        last_page_number = weights[country]
        random_page = random.randint(1, last_page_number)
        
        page_url = f"https://boardgamegeek.com/users/page/{random_page}?country={country}&state=&city=&avatar=1"
        response = requests.get(page_url)
        if response.status_code == 200:
            page_soup = BeautifulSoup(response.text, 'html.parser')
            
            user_divs = page_soup.find_all('div', class_='username')
            for div in user_divs:
                username = div.find('a').text
                if username not in local_usernames:
                    local_usernames.append(username)
                    if len(local_usernames) >= 100000:
                        break
        time.sleep(0.5)  # Adjust delay to prevent overloading the server
    return local_usernames

def write_usernames_to_file(local_usernames):
    global total_usernames_acquired
    with lock:
        with open('username2s.txt', 'a') as file:
            for username in local_usernames:
                if username not in usernames:
                    usernames.append(username)
                    file.write(username + '\n')
        total_usernames_acquired += len(local_usernames)
        print(f'Total usernames acquired: {total_usernames_acquired}')

# Number of parallel tasks to run
n_tasks = 500  # This is an example; adjust based on your needs

# Parallel execution
results = Parallel(n_jobs=5)(delayed(fetch_usernames)() for _ in range(n_tasks))

# Write results to file
for result in results:
    write_usernames_to_file(result)

print("Completed writing usernames.")


KeyboardInterrupt: 

In [8]:
import random
import requests
from bs4 import BeautifulSoup
from joblib import Parallel, delayed
import time

def fetch_usernames(country, random_page):
    usernames = []
    page_url = f"https://boardgamegeek.com/users/page/{random_page}?country={country}&state=&city=&avatar=1"
    response = requests.get(page_url)
    page_soup = BeautifulSoup(response.text, 'html.parser')
    
    # Extract usernames
    user_divs = page_soup.find_all('div', class_='username')
    for div in user_divs:
        username = div.find('a').text
        usernames.append(username)
    
    return usernames

# Function to generate random pages for a country and fetch usernames
def process_country(country):
    last_page_number = weights[country]
    random_page = random.randint(1, last_page_number)
    return fetch_usernames(country, random_page)

def main():
    #read countries.json
    weights = json.load(open('countries.json'))
    country_list = list(weights.keys())
    weightsC = list(weights.values())

    num_batches = 5
    target_usernames_per_batch = 100000
    total_usernames_needed = num_batches * target_usernames_per_batch
    
    for batch_number in range(1, num_batches + 1):
        usernames = set()
        
        while len(usernames) < target_usernames_per_batch:
            selected_countries = random.choices(country_list, weights=weightsC, k=10)
            
            # Parallelize the fetching of usernames
            results = Parallel(n_jobs=5)(delayed(process_country)(country) for country in selected_countries)
            
            for result in results:
                for username in result:
                    if username not in usernames:
                        usernames.add(username)
                        if len(usernames) >= target_usernames_per_batch:
                            break

            print(f"Batch {batch_number}: Added usernames:", len(usernames))
            
            # Wait a bit before making another request
            time.sleep(1)
        
        # Write usernames to batch-specific file
        with open(f'usernames_batch_{batch_number}.txt', 'w') as file:
            for username in usernames:
                file.write(username + '\n')

if __name__ == "__main__":
    main()


Batch 1: Added usernames: 0
Batch 1: Added usernames: 0
Batch 1: Added usernames: 0
Batch 1: Added usernames: 0
Batch 1: Added usernames: 25
Batch 1: Added usernames: 25
Batch 1: Added usernames: 25
Batch 1: Added usernames: 25
Batch 1: Added usernames: 25
Batch 1: Added usernames: 25
Batch 1: Added usernames: 25
Batch 1: Added usernames: 25
Batch 1: Added usernames: 25


KeyboardInterrupt: 

In [21]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import random

In [45]:
base_url = "https://boardgamegeek.com/xmlapi2/user?name=Bradsmith488&buddies=1&guilds=1"
response = requests.get(base_url)
print(response.status_code)
if response.status_code == 200:
    user = ET.fromstring(response.text)
    
    if user is not None:
        lastlogin = user.find('lastlogin').get('value')
        guilds = user.find('guilds')
        guild_total = int(guilds.get('total')) if guilds is not None else 0
        
        # Check conditions
        if (lastlogin.startswith('2023') or lastlogin.startswith('2024')) and guild_total >= 1:
            id = user.get('id')
            yearregistered = user.find('yearregistered').get('value')
            country = user.find('country').get('value')
            buddies = user.find('buddies')
            geekfriends_total = buddies.get('total') if buddies is not None else 0
            geekfriends = [b.get('name') for b in buddies.findall('buddy')] if buddies is not None else []
            guilds_list = [g.get('id') for g in guilds.findall('guild')] if guilds is not None else []
            print(id, yearregistered, country, geekfriends_total, geekfriends, guild_total, guilds_list)
            

200
932944 2014 United States 18 ['Bobdole04126623', 'Cole Wehrle', 'DCMann2', 'DeagleSeagull', 'ggambill', 'HiveGod', 'jadon79', 'jmdsplotter', 'Jonahsimmons15', 'joriswiersinga', 'kuhrusty', 'LaurenElainey', 'loftis77', 'phileklund', 'Sluggs', 'Thackerrst', 'thebadbishop', 'TheFishofDoom'] 4 ['1290', '2044', '2511', '3098']


In [25]:
names = set()
with open('username2s.txt','r') as file:
    file=file.readlines()
    for line in file:
        line = line.strip()
        names.add(line)



33344

In [31]:
len(names)

33344

In [45]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import random

# Example array of usernames

aux = random.sample(sorted(names), 33000) 



# Base URL for the API (this needs to be updated to the correct API endpoint if different)

# DataFrame to hold all user data
columns = ['id', 'username', 'yearregistered', 'lastlogin', 'country', 'geekfriends', 'geekfriends_total', 'guilds', 'guild_total']
df = pd.DataFrame(columns=columns)

# Function to fetch and parse user data
def fetch_user_data(username):
    try:
        url = f"https://boardgamegeek.com/xmlapi2/user?name={username}&buddies=1&guilds=1"
        response = requests.get(url)
        if response.status_code == 200:
            user = ET.fromstring(response.content)
            if user is not None:
                lastlogin = user.find('lastlogin').get('value')
                guilds = user.find('guilds')
                guild_total = int(guilds.get('total')) if guilds is not None else 0
                
                # Check conditions
                if (lastlogin.startswith('2023') or lastlogin.startswith('2024') or lastlogin.startswith('2021') or lastlogin.startswith('2022')) and guild_total >= 1:
                    id = user.get('id')
                    yearregistered = user.find('yearregistered').get('value')
                    country = user.find('country').get('value')
                    buddies = user.find('buddies')
                    geekfriends_total = buddies.get('total') if buddies is not None else 0
                    geekfriends = [b.get('name') for b in buddies.findall('buddy')] if buddies is not None else []
                    guilds_list = [g.get('id') for g in guilds.findall('guild')] if guilds is not None else []
                    
                    return {
                        'id': id,
                        'username': username,
                        'yearregistered': yearregistered,
                        'lastlogin': lastlogin,
                        'country': country,
                        'geekfriends': geekfriends,
                        'geekfriends_total': geekfriends_total,
                        'guilds': guilds_list,
                        'guild_total': guild_total
                    }
        else:
            print(f"Failed to retrieve data for {username}, HTTP status code: {response.status_code}")
    except Exception as e:
        print(f"Error processing {username}: {e}")
    return None

data = []
count = 0
# Process each username
for username in aux:
    user_data = fetch_user_data(username)
    if user_data:
        data.append(user_data)
        print(f'\n{count} Added to df\n')
    if count == 33000:
        break
    time.sleep(random.uniform(2,3))
    count = count + 1
    print(f"{count}")

df = pd.DataFrame(data, columns=columns)



# Display the DataFrame



0 Added to df

1

1 Added to df

2
3

3 Added to df

4
5
6
7

7 Added to df

8
9

9 Added to df

10

10 Added to df

11

11 Added to df

12

12 Added to df

13
14
15
16
17
18

18 Added to df

19
20
21

21 Added to df

22
23
24
25

25 Added to df

26
Error processing DoctorStabby: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
27
28
29
30
31
32
33
34

34 Added to df

35

35 Added to df

36
37
38

38 Added to df

39
40
41

41 Added to df

42
43
44
45

45 Added to df

46

46 Added to df

47

47 Added to df

48
49
50
51
52
53
54

54 Added to df

55

55 Added to df

56

56 Added to df

57
58
59
60
61
62
63
64

64 Added to df

65
66
67
68
69

69 Added to df

70
71
72
73
74
75

75 Added to df

76
77

77 Added to df

78
79
80
81
82
83
84

84 Added to df

85
86
87

87 Added to df

88

88 Added to df

89
90
91
92
93
94
95
96
97
98
99

99 Added to df

100

100 Added to df

101

101 Added to df

102
103
104

104 Added to df

105
106

106 Added to df



KeyboardInterrupt: 

In [47]:
df = pd.DataFrame(data, columns=columns)

In [60]:
filt = df['guild_total'] >= 1
filt2 = df['geekfriends_total'] >= 1
ex = df[filt | filt2]
ex

,id,username,yearregistered,lastlogin,country,geekfriends,geekfriends_total,guilds,guild_total
3,335267,pcservbot,2010,2022-10-21,United States,"[bop517, Cate108, cosine, fathergeek, galderon...",10,"[193, 198, 674]",3
11,1662322,grog3514,2017,2024-06-30,United States,"[Chris Schreiber, DogVood, gpioski, KYGameMast...",5,[],0
19,3451954,Tabodi,2023,2023-07-14,Finland,"[Blue_rose, Plush]",2,[],0
20,1039070,Yender,2015,2024-06-28,Romania,[],0,[1042],1
21,2792629,yummTwinkies,2020,2024-06-07,United States,[daddylampshade],1,[],0
...,...,...,...,...,...,...,...,...,...
2904,447286,Hightree,2011,2024-06-23,Netherlands,"[Phreakyhamster, topstefan]",2,[],0
2906,2067383,Proti_BG,2019,2024-06-29,Romania,"[dukepuccini, Tibygod]",2,[4293],1
2915,472157,Webgears,2012,2024-03-22,Denmark,"[Tazzi, TroelsR]",2,[],0
2918,108919,RexDart817,2006,2024-06-30,United States,[Chrisxny],1,"[160, 175, 392]",3


In [64]:
ex.to_csv('players3.csv', index=False)

In [65]:
df2 = pd.read_csv('second_order.csv')


In [66]:
df2

,id_x,username,yearregistered,lastlogin,country,geekfriends,geekfriends_total,guilds,guild_total,id_y,...,Geek_rating,rank,num_voters,type,category,playing_time,first_order_guilds,first_order_geeks,owns,second_order_guilds
0,335267,pcservbot,2010,2022-10-21,United States,"['bop517', 'Cate108', 'cosine', 'fathergeek', ...",10,"['193', '198', '674']",3,13,...,6.92136,555,127892,strategygames,Economic,120,55.932203,0.0,0,46.590909
1,335267,pcservbot,2010,2022-10-21,United States,"['bop517', 'Cate108', 'cosine', 'fathergeek', ...",10,"['193', '198', '674']",3,822,...,7.29668,224,127130,familygames,Medieval,45,47.457627,0.0,0,52.272727
2,335267,pcservbot,2010,2022-10-21,United States,"['bop517', 'Cate108', 'cosine', 'fathergeek', ...",10,"['193', '198', '674']",3,30549,...,7.42958,147,125684,strategygames,Medical,45,57.627119,0.0,0,54.545455
3,335267,pcservbot,2010,2022-10-21,United States,"['bop517', 'Cate108', 'cosine', 'fathergeek', ...",10,"['193', '198', '674']",3,68448,...,7.57338,91,104710,strategygames,Ancient,30,47.457627,0.0,0,39.772727
4,335267,pcservbot,2010,2022-10-21,United States,"['bop517', 'Cate108', 'cosine', 'fathergeek', ...",10,"['193', '198', '674']",3,167791,...,8.21238,7,99143,strategygames,Economic,120,32.203390,0.0,0,22.727273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
990,2792629,yummTwinkies,2020,2024-06-07,United States,['daddylampshade'],1,[],0,169426,...,7.23799,263,20556,strategygames,Dice,90,0.000000,0.0,0,0.000000
991,2792629,yummTwinkies,2020,2024-06-07,United States,['daddylampshade'],1,[],0,5,...,7.12428,343,20518,strategygames,Economic,90,0.000000,0.0,0,0.000000
992,2792629,yummTwinkies,2020,2024-06-07,United States,['daddylampshade'],1,[],0,175914,...,7.80202,39,20456,strategygames,Economic,240,0.000000,0.0,0,0.000000
993,2792629,yummTwinkies,2020,2024-06-07,United States,['daddylampshade'],1,[],0,216132,...,7.68279,64,20424,strategygames,Economic,120,0.000000,0.0,0,0.000000
